## Maximum bidding vs. average bidding

Facebook recently introduced a new type of bid, average bidding, as an alternative to the current type of bidding called maximum bidding.

One of our customers decided to test this new feature and wants to do an A/B test to see if average bidding converts more than maximum bidding.

### Story of Dataset 
In this dataset, which includes the customer's website information, there is information such as the number of advertisements that users see and click, as well as earnings information from here.


There are two separate data sets, the control and test groups.

### Variables
Impression – Ad views
Clicks - Indicates the number of clicks on the displayed ad.
Purchase – Indicates the number of products purchased after the ads clicked
Earning – Earnings after purchased products

In [1]:
import pandas as pd
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu, pearsonr, spearmanr, kendalltau

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
pd.set_option('display.float_format', lambda x: '%.5f' % x)


In [2]:
df_control = pd.read_excel(r'/Users/firatsoydinc/Desktop/Miuul/Week - 5/Ders Oncesi Notlar/ab_testing.xlsx',sheet_name= 'Control Group')
df_test  = pd.read_excel(r'/Users/firatsoydinc/Desktop/Miuul/Week - 5/Ders Oncesi Notlar/ab_testing.xlsx',sheet_name= 'Test Group')

### Churn_rate variable that shows after how many ad views the customer clicked on that ad 

In [3]:
df_control['churn_rate'] = df_control['Impression']/df_control['Click']
df_test['churn_rate']= df_test['Impression']/ df_test['Click']


### Data Cleaning

In [4]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit


In [5]:
replace_with_thresholds(df_control,'churn_rate')
replace_with_thresholds(df_test,'churn_rate')

### A/B Test
A/B test is performed to check whether there is a statistically significant difference between two independent data sets. There are 2 different A/B test methods. One of them is parametric and the other is nonparametric. In order to perform a parametric test, the data set must provide 3 assumptions. If even one of these assumptions is not met, non-parametric testing is applied.

Assumptions: 
    
    Number of observations must be at least 30  (n>30)
    
    Variables must be normal distributed 
    
    Variance homogeneity must be assigned by variables

#### First Assumption n>30

In [6]:
print('Control data set has',df_control.size, 'observations.')
print('Test data set has ',df_test.size,'observations.')

Control data set has 200 observations.
Test data set has  200 observations.


#### Second Assumption (Normal Distribution)

    H1: Variables are normally distributed.
    H0: Variables are not normally distributed.
    
If p_value is less than 0.05 h1 is reject, else fail to reject 

In [7]:
for each in df_control.columns:
    test_stat, pvalue = shapiro(df_control[each])
    print('Test Stat of ', each ,'= %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat of  Impression = 0.9697, p-value = 0.3514
Test Stat of  Click = 0.9844, p-value = 0.8461
Test Stat of  Purchase = 0.9773, p-value = 0.5891
Test Stat of  Earning = 0.9756, p-value = 0.5306
Test Stat of  churn_rate = 0.9548, p-value = 0.1107


Variables of control dataset' p value is greater than 0.05. So H1 hypotesis is fail to reject. Variables are normally distributed.

In [8]:
for each in df_test.columns:
    test_stat, pvalue = shapiro(df_test[each])
    print('Test Stat of ', each ,'= %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat of  Impression = 0.9720, p-value = 0.4148
Test Stat of  Click = 0.9896, p-value = 0.9699
Test Stat of  Purchase = 0.9589, p-value = 0.1541
Test Stat of  Earning = 0.9780, p-value = 0.6163
Test Stat of  churn_rate = 0.9161, p-value = 0.0058


Variables of test dataset' p value is greater than 0.05 except churn_rate variable. If A/B testing is to be applied to the churn rate variable, non-parametric testing should be applied. If it will be applied to other variables, parametric test assumptions should continue to be checked for those variables.

#### Third Assumption (Variance Homogenity)

    H1: Variances of Variables Are Homogeneously Distributed
    H0: Not homogeneously distributed
   
If p_value is less than 0.05 h1 is reject, else fail to reject 

In [9]:
for each1 in df_control.columns:
    test_stat, pvalue = levene(df_control[each1],df_test[each1])
    print('Test Stat of ', each1 ,'= %.4f, p-value = %.4f' % (test_stat, pvalue))


Test Stat of  Impression = 0.5865, p-value = 0.4461
Test Stat of  Click = 6.3041, p-value = 0.0141
Test Stat of  Purchase = 2.6393, p-value = 0.1083
Test Stat of  Earning = 0.3532, p-value = 0.5540
Test Stat of  churn_rate = 0.7336, p-value = 0.3943


### Parametric Test 

    H1: There is a significant difference between the data
    H0: There is no significant difference between the data

In [10]:
parametric_test_variables = ['Impression','Click','Purchase','Earning']

for each in parametric_test_variables:
    test_stat, pvalue = ttest_ind(df_control[each],
                               df_test[each],
                                  equal_var=True)
    print('Test Stat of ', each ,'= %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat of  Impression = -4.2966, p-value = 0.0000
Test Stat of  Click = 4.4266, p-value = 0.0000
Test Stat of  Purchase = -0.9416, p-value = 0.3493
Test Stat of  Earning = -9.2545, p-value = 0.0000


Impression, click, and earnings variables have a p-value less than 0.05. There is a statistical difference between these groups of variables. But since the p_value is greater than 0.05 in the purchase variable, there is no statistical difference.

### Non-Parametric Test 

    H1: There is a significant difference between the data
    H0: There is no significant difference between the data

In [11]:
test_stat, pvalue = mannwhitneyu(df_control['churn_rate'],
                               df_test['churn_rate'])

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 292.0000, p-value = 0.0000


Since the p value of the Churn rate variable is less than 0.05, we can say that there is a statistical difference between them. But since there is no difference in the purchase variable, it will be more efficient if our customer continues to use the average bidding method. Statistical controls can be made again when the number of data for this study is increased.